In [1]:
from utils.imports import *

Using TensorFlow backend.


In [2]:
def get_net(input_shape, load_weight_path=None, features=False, mal=False):
    width = 64
    inputs = Input(shape=(1, 32, 32, 32), name="input_1")
    x = inputs
    x = AveragePooling3D(pool_size=(2, 1, 1), strides=(2, 1, 1), border_mode="same")(x)
    x = Convolution3D(width, 3, 3, 3, activation='relu', border_mode='same', name='conv1', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), border_mode='valid', name='pool1')(x)

    # 2nd layer group
    x = Convolution3D(width*2, 3, 3, 3, activation='relu', border_mode='same', name='conv2', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool2')(x)
    x = Dropout(p=0.3)(x)

    # 3rd layer group
    x = Convolution3D(width*4, 3, 3, 3, activation='relu', border_mode='same', name='conv3a', subsample=(1, 1, 1))(x)
    x = Convolution3D(width*4, 3, 3, 3, activation='relu', border_mode='same', name='conv3b', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool3')(x)
    x = Dropout(p=0.4)(x)

    # 4th layer group
    x = Convolution3D(width*8, 3, 3, 3, activation='relu', border_mode='same', name='conv4a', subsample=(1, 1, 1))(x)
    x = Convolution3D(width*8, 3, 3, 3, activation='relu', border_mode='same', name='conv4b', subsample=(1, 1, 1),)(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool4')(x)
    x = Dropout(p=0.5)(x)
       
    last64 = Convolution3D(64, 2, 2, 2, activation="relu", name="last_64")(x)
    out_class = Convolution3D(1, 1, 1, 1, activation="softmax", name="out_class_last")(last64)
    out_class = Flatten(name="out_class")(x)
    
    out_class = Dense(2)(out_class) 
    out_class = Activation('softmax')(out_class)
    
    model = Model(input=inputs, output=out_class)
    model.compile(optimizer=SGD(lr=1e-3, momentum=0.9, nesterov=True), loss='categorical_crossentropy', metrics=['accuracy'])
    #model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])   
    return model

In [3]:
output_path = PATH['cls_train_cube_30']
output_true = PATH['cls_train_cube_30_true']
output_false = PATH['cls_train_cube_30_false']
model_paths = PATH['model_paths']
model_final = PATH['model_final']

In [ ]:
def get_dirfiles(dir):
    file_list = []
    subset_path = os.listdir(dir)
    for _ in range(len(subset_path)):
        if subset_path[_] != '.DS_Store':
            file_list.append(dir + subset_path[_])
    return file_list

def train_generator(output_true,output_false):
    train_nb = 21000    
    file_list_true = get_dirfiles(output_true)
    file_list_false = get_dirfiles(output_false)        
    file_list_true = np.random.choice(file_list_true, train_nb)
    file_list_false = np.random.choice(file_list_false, train_nb)    
    nb_true = len(file_list_true) + len(file_list_false)    
    sample = np.zeros([nb_true,32, 32, 32])
    labels = np.zeros([nb_true,2])
    for i in tqdm(range(len(file_list_true))):       
        cc= np.load(file_list_true[i])
        sample[i] = cc
        labels[i] = [0.,1.]
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_false[j])
        sample[j+len(file_list_true)] = bb 
        labels[j+len(file_list_true)] = [1.,0.]
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels


def fenlei_fit(name, load_check = False,batch_size=2, epochs=100,check_name = None):
    t = time.time()
    callbacks = [EarlyStopping(monitor='val_loss', patience = 8, verbose = 1),
                 ModelCheckpoint((model_paths + '{}.h5').format(name),
                                 monitor='val_loss',
                                 verbose = 0,
                                 save_best_only = True)]
    if load_check:
        check_model = (model_paths + '{}.h5').format(check_name)
        model = load_model(check_model)
    else:
        #model = classifier((1, 32, 32, 32),128,(3, 3, 3), (2, 2, 2))
        model = get_net((1, 32, 32, 32))
    x,y = train_generator(output_true,output_false)
    model.fit(x,y, batch_size=batch_size, epochs=epochs,
              validation_split=0.2,verbose=1, callbacks=callbacks, shuffle=True) 
    return model

In [ ]:
fenlei_fit('Fenge_32_32_32_0629_2', load_check = True, batch_size=8, epochs=200, check_name = 'Fenge_32_32_32_0629')

100%|██████████| 21000/21000 [05:20<00:00, 65.54it/s]


Train on 33600 samples, validate on 8400 samples
Epoch 1/200
33600/33600 [==============================] - 669s - loss: 0.5832 - acc: 0.6899 - val_loss: 0.5168 - val_acc: 0.8102
Epoch 2/200
33600/33600 [==============================] - 619s - loss: 0.3499 - acc: 0.8445 - val_loss: 0.1978 - val_acc: 0.9502
Epoch 3/200
33600/33600 [==============================] - 615s - loss: 0.2070 - acc: 0.9167 - val_loss: 0.1518 - val_acc: 0.9615
Epoch 4/200
33600/33600 [==============================] - 620s - loss: 0.1390 - acc: 0.9469 - val_loss: 0.1308 - val_acc: 0.9631
Epoch 5/200
33600/33600 [==============================] - 694s - loss: 0.1031 - acc: 0.9626 - val_loss: 0.0913 - val_acc: 0.9786
Epoch 6/200
33600/33600 [==============================] - 665s - loss: 0.0777 - acc: 0.9720 - val_loss: 0.1060 - val_acc: 0.9663
Epoch 7/200
33600/33600 [==============================] - 664s - loss: 0.0614 - acc: 0.9781 - val_loss: 0.0848 - val_acc: 0.9789
Epoch 8/200
33600/33600 [================

In [ ]:
file_list_true = get_dirfiles(output_true)
file_list_false = get_dirfiles(output_false)

In [ ]:
#model_pred = classifier((1, 36, 36, 36), (3, 3, 3), (2, 2, 2))
model_pred = load_model(model_paths + 'Fenge_32_32_32_0626.h5')

In [ ]:
cc = []
for i in file_list_false[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0
for i in cc:
    if i[0][0] > 0.9:
        count += 1
print count*1.0/len(cc)

In [ ]:
cc

In [ ]:
cc = []
for i in file_list_true[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0


In [ ]:
for i in cc:
    if i[0][1] > 0.9:
        count += 1
print count*1.0/len(cc)

In [ ]:
cc

In [ ]:
len(file_list_false)